#Importar librerias

In [ ]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=c0f4017ce659861c67263b4e976c1171e6f64f779a7907569eafa757380e9234
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [ ]:
import requests
import re
import csv
from bs4 import BeautifulSoup
import time
import pickle
import os
import pandas as pd
import spacy
from langdetect import detect, DetectorFactory

In [ ]:
!python -m spacy download es_core_news_sm

2023-12-20 17:36:47.726616: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-20 17:36:47.726677: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-20 17:36:47.729462: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-20 17:36:49.371939: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 39.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Ajustables

In [ ]:
# Encabezado de las solicitudes http
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

# Generos y paginas correspondientes a la página de Gandhi
generos_num_paginas =[("ensayo-literario", 11),
          ("ciencia-ficcion", 4),
          ("terror-y-suspenso", 8),
          ("arte-ii", 14),
          ("arquitectura", 3),
          ("cine", 6),
          ("fotografia", 6),
          ("desarrollo-humano", 8),
          ("salud-y-ejercicio", 5),
          ("negocios-y-finanzas", 4),
          ("derecho", 14),
          ("psicologia-psicoanalisis-psiquiatra", 2),
          ("filosofia", 17),
          ("viajes", 3),
          ("gastronomia", 3)]

ruta_libros = '/content/drive/MyDrive/ESCOM/8vo Semestre/NLP/libros.csv'
ruta_enlaces_invalidos = '/content/drive/MyDrive/ESCOM/8vo Semestre/NLP/enlaceinvalido.csv'
ruta_libros_limpiado = '/content/drive/MyDrive/ESCOM/8vo Semestre/NLP/librosLimpiado.csv'
ruta_libros_limpiado_esp = '/content/drive/MyDrive/ESCOM/8vo Semestre/NLP/librosLimpiadoEN.csv'

# Obtención de enlaces de libros

In [ ]:
urls_grid = []
enlaces_libros = set()

# Genera los enlaces de las paginas de cada genero para hacer el web scrapping
urls_grid = [(f'https://www.gandhi.com.mx/libros/{genero}?p={num_pagina}', genero) for genero, num_paginas in generos_num_paginas for num_pagina in range(1, num_paginas + 1)]


for url_g, genero in urls_grid:
    response = requests.get(url_g, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Buscar todos los elementos 'a' y extraer el atributo 'href'
    for enlace in soup.find_all('a', href=True):
        aux = (enlace['href'], genero)
        enlaces_libros.add(aux)
        print(aux)

    time.sleep(1)

# La expresión regular que coincide con URLs de Gandhi sin ruta adicional
# pattern = r"https://www.gandhi.com.mx/[^/]+$"
pattern = r"https://www.gandhi.com.mx/.+"

# Filtrar las URLs que coinciden con la expresión regular
enlaces_libros = [(url, genero) for url, genero in enlaces_libros if re.match(pattern, url)]

# for enlace in enlaces_libros:
#   print(enlace)

In [ ]:
len(enlaces_libros)

9528

##Guardar los enlaces de los libros en un .pkl

In [ ]:
with open('enlaces_libros.pkl', 'wb') as file:
    pickle.dump(enlaces_libros, file)

##Cargar los enlaces de los libros desde un .pkl

In [ ]:
with open('enlaces_libros.pkl', 'rb') as file:
    enlaces_libros = pickle.load(file)

# Scrapping individual

In [ ]:
enlace_libro = "https://www.gandhi.com.mx/pasiones-toxicas"
response = requests.get(enlace_libro, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# Extraer el ISBN
ISBN_tag = soup.find('div', class_='isbn').find_all('span')[1]
ISBN = ISBN_tag.get_text(strip=True) if ISBN_tag else ''

# Extraer el título
titulo_tag = soup.find('h1', class_='page-title')
titulo = titulo_tag.get_text(strip=True) if titulo_tag else ''

# Extraer el nombre del autor
autor_tag = soup.find('div', class_='autor').find('a')
autor = autor_tag.get_text(strip=True) if autor_tag else ''

# Extraer la editorial
editorial_tag = soup.find('div', class_='editoriales').find('a')
editorial = editorial_tag.get_text(strip=True) if editorial_tag else ''

# Extraer el número de páginas
num_paginas_tag = soup.find('li', {'data-li': 'Número de páginas'})
num_paginas = num_paginas_tag.find('span', class_='attr-data').text if num_paginas_tag else ''

# Extraer la fecha de publicación
fecha_publicacion_tag = soup.find('li', {'data-li': 'Fecha de publicación'})
fecha_publicacion = fecha_publicacion_tag.find('span', class_='attr-data').text if fecha_publicacion_tag else ''

# Extraer la sinopsis
sinopsis_tag = soup.find('div', id='description')
sinopsis = sinopsis_tag.get_text(strip=True) if sinopsis_tag else ''

img_tag = soup.find('img', class_='product-photo')
image_url = img_tag.get('data-srclazy') or img_tag.get('data-lazy') if img_tag else ''


print([ISBN, titulo, autor, editorial, num_paginas, fecha_publicacion, sinopsis, image_url, enlace_libro])

if all([ISBN, titulo, autor, editorial, num_paginas, fecha_publicacion, sinopsis, image_url, enlace_libro]):

            print([ISBN, titulo, autor, editorial, num_paginas, fecha_publicacion, sinopsis, image_url, enlace_libro])

['9786075692180', 'Pasiones tóxicas', 'Bernardo Stamateas', 'Paidós', '272', ' 20 de Mayo 2022 ', 'Pasiones tóxicas Bernardo Stamateas Todas las parejas pasan por crisis, el tema es tener los recursos necesarios para atravesarlas. Porque cuando permanecemos atrapados y detenidos en este estado, los conflictos y la confusión llegarán a nosotros de manera inevitable. Por eso, en este libro analizaremos las pasiones más frecuentes de las parejas: Las pasiones del engaño y la infidelidad. Las pasiones de la posesividad. Las pasiones del estancamiento. Las pasiones de la competitividad. Las pasiones de la descalificación. La idea de Pasiones tóxicas es dar herramientas para repensar y activar los recursos que ya están adentro nuestro. Y es así como el licenciado Bernardo Stamateas echa abajo varios mitos que responden a idealizaciones románticas de la pareja, restituyendo ideas y conceptos más humanos: el conflicto y el enojo son normales, en una pareja nadie tiene la razón, estar casado es

# Scrapping por lotes

## Obtecion de enlaces faltantes

In [ ]:
libros_existentes = set()
try:
    with open(ruta_libros, mode='r', newline='', encoding='utf-8') as file:
        reader = csv.reader(file)
        next(reader)  # Saltar la cabecera
        for fila in reader:
            tupla = (fila[9], fila[2])
            libros_existentes.add(tupla)
except FileNotFoundError:
    print("Archivo no encontrado, se creará uno nuevo.")

In [ ]:
enlaces_invalidos = set()
try:
    with open(ruta_enlaces_invalidos, mode='r', newline='', encoding='utf-8') as file:
        reader = csv.reader(file)
        next(reader)  # Saltar la cabecera
        for fila in reader:
            tupla = (fila[0], fila[1])
            enlaces_invalidos.add(tupla)
except FileNotFoundError:
    print("Archivo no encontrado, se creará uno nuevo.")

In [ ]:
print("enlaces_libros: ",len(enlaces_libros))
print("enlaces_invalidos: ",len(enlaces_invalidos))
print("libros_existentes: ",len(libros_existentes))
enlaces_libros = set(enlaces_libros)
enlaces_libros = enlaces_libros.difference(libros_existentes)
enlaces_libros = enlaces_libros.difference(enlaces_invalidos)
print(len(enlaces_libros))

enlaces_libros:  993
enlaces_invalidos:  6827
libros_existentes:  2701
0


## Extracción de datos

In [ ]:
def extraer_texto(soup, selector, attribute=None, all_spans=False, index=0):
    tag = soup.select_one(selector)
    if tag and all_spans:
        spans = tag.find_all('span')
        if spans and len(spans) > index:
            return spans[index].get_text(strip=True)
    return tag.get_text(strip=True) if tag else ''

In [ ]:
file_exist = os.path.exists(ruta_libros)
enlaces_error = []

with open(ruta_libros, mode='a', newline='', encoding='utf-8') as file:
    with open(ruta_enlaces_invalidos, mode='a', newline='', encoding='utf-8') as enlaces_file:
      writer = csv.writer(file)
      writer_enalces = csv.writer(enlaces_file)

      if not file_exist:
          writer.writerow(['ISBN', 'Título', 'Género', 'Autor', 'Editorial', 'Número de Páginas', 'Fecha de Publicación', 'Sinopsis', "Portada", "URL"])

      for enlace_libro, genero in enlaces_libros:
          response = requests.get(enlace_libro, headers=headers)
          soup = BeautifulSoup(response.text, 'html.parser')

          try:
            ISBN = extraer_texto(soup, 'div.isbn', all_spans=True, index=1)
            titulo = extraer_texto(soup, 'h1.page-title')
            autor = extraer_texto(soup, 'div.autor a')
            editorial = extraer_texto(soup, 'div.editoriales a')
            num_paginas = extraer_texto(soup, 'li[data-li="Número de páginas"] .attr-data')
            fecha_publicacion = extraer_texto(soup, 'li[data-li="Fecha de publicación"] .attr-data')
            sinopsis = extraer_texto(soup, 'div#description')

            img_tag = soup.find('img', class_='product-photo')
            image_url = img_tag.get('data-srclazy') or img_tag.get('data-lazy') if img_tag else ''

            if all([ISBN, titulo, genero, autor, editorial, num_paginas, fecha_publicacion, sinopsis, image_url, enlace_libro]):
                writer.writerow([ISBN, titulo, genero, autor, editorial, num_paginas, fecha_publicacion, sinopsis, image_url, enlace_libro])
                print([ISBN, titulo, genero, autor, editorial, num_paginas, fecha_publicacion, sinopsis, image_url, enlace_libro])
            else:
                writer_enalces.writerow([enlace_libro, genero])
                print("Enlace invalido: ", enlace_libro)
          except Exception as e:
            print("Error al procesar la página:", enlace_libro, "\nError:", e)
          # time.sleep(0.05)

print("Datos guardados en libros.csv")


# Limpieza de los datos

In [ ]:
# Carga el archivo CSV
df = pd.read_csv(ruta_libros)

# Reemplaza 'ciencia-ficcion' por 'Ciencia ficción' en la columna deseada
df['Género'] = df['Género'].str.replace('arquitectura', 'Arquitectura')
df['Género'] = df['Género'].str.replace('arte-ii', 'Arte')
df['Género'] = df['Género'].str.replace('ciencia-ficcion', 'Ciencia Ficción')
df['Género'] = df['Género'].str.replace('cine', 'Cine')
df['Género'] = df['Género'].str.replace('derecho', 'Derecho')
df['Género'] = df['Género'].str.replace('desarrollo-humano', 'Desarrollo Humano')
df['Género'] = df['Género'].str.replace('ensayo-literario', 'Ensayo Literario')
df['Género'] = df['Género'].str.replace('filosofia', 'Filosofía')
df['Género'] = df['Género'].str.replace('fotografia', 'Fotografía')
df['Género'] = df['Género'].str.replace('gastronomia', 'Gastronomía')
df['Género'] = df['Género'].str.replace('negocios-y-finanzas', 'Negocios y Finanzas')
df['Género'] = df['Género'].str.replace('psicologia-psicoanalisis-psiquiatra', 'Psicología Psicoanalisis Psiquiatra')
df['Género'] = df['Género'].str.replace('salud-y-ejercicio', 'Salud y Ejercicio')
df['Género'] = df['Género'].str.replace('terror-y-suspenso', 'Terror y Suspenso')
df['Género'] = df['Género'].str.replace('viajes', 'Viajes')

In [ ]:
nlp = spacy.load("es_core_news_sm")

In [ ]:
def normalizacion (cadena):
  doc = nlp(cadena)
  texto_normalizado = ""
  for token in doc:
    if(not token.is_stop and not token.pos_=='SPACE'):
      texto_normalizado += f" {token.lemma_}"
      print(token.text, token.pos_, token.dep_, token.lemma_)

  return texto_normalizado[1:]

In [ ]:
df['Normalización'] = df.apply(lambda fila: normalizacion(f"{fila['Título']} {fila['Género']} {fila['Autor']} {fila['Editorial']} {fila['Fecha de Publicación']} {fila['Sinopsis']}"), axis=1)

In [ ]:
df = pd.read_csv(ruta_libros_limpiado)
DetectorFactory.seed = 0

df['Idioma'] = df['Normalización'].apply(detect)
df = df[df['Idioma'] == 'es']
df = df.drop(['Idioma'], axis=1)

In [ ]:
# Guarda el archivo CSV modificado
df.to_csv(ruta_libros_limpiado, index=False)